In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')
import pickle

import numpy as np
import pandas as pd
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_curve

## Data

In [3]:
# saved from infCutoff.ipynb
with open('./data/split_data.pkl', 'rb') as f:
    d = pickle.load(f)
    train_set, calibration_set, test_set = d['train_set'], d['calibration_set'], d['test_set']
train_set.shape, calibration_set.shape, test_set.shape

((48000, 102), (28000, 102), (4000, 102))

In [4]:
X_calib, nu_calib, y_calib = calibration_set.drop(columns=['run_name', 'cell_type']).values, calibration_set.loc[:, 'run_name'].values, calibration_set.loc[:, 'cell_type'].values.astype(int)
X_calib.shape, nu_calib.shape, y_calib.shape

((28000, 100), (28000,), (28000,))

In [5]:
# nu test not used. mu test only to check results
X_test, nu_test, y_test = test_set.drop(columns=['run_name', 'cell_type']).values, test_set.loc[:, 'run_name'].values, test_set.loc[:, 'cell_type'].values.astype(int)
X_test.shape, nu_test.shape, y_test.shape

((4000, 100), (4000,), (4000,))

## Classifier

In [6]:
# saved from infCutoff.ipynb
with open('./results/pickles/catgb_clf.pkl', 'rb') as f:
    cat_gb = pickle.load(f)

In [7]:
calibrated_cat_gb = CalibratedClassifierCV(
    estimator=cat_gb,
    method='isotonic',
    cv='prefit'
)
calibrated_cat_gb.fit(X=X_calib, y=y_calib)

CalibratedClassifierCV(cv='prefit',
                       estimator=RandomizedSearchCV(cv=5,
                                                    estimator=<catboost.core.CatBoostClassifier object at 0x7f1badaeae50>,
                                                    n_iter=25, n_jobs=20,
                                                    param_distributions={'depth': [1,
                                                                                   3,
                                                                                   5,
                                                                                   7,
                                                                                   10],
                                                                         'iterations': [100,
                                                                                        300,
                                                                                        500,
                                                                                        700,
                                                                                        1000]},
                                                    verbose=1),
                       method='isotonic')

In [7]:
alpha_grid = np.array([0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.05, 0.01])

### Class-specific cutoffs

In [9]:
probs_calib = calibrated_cat_gb.predict_proba(X=X_calib)
fpr0, tpr0, thresholds0 = roc_curve(y_true=y_calib, y_score=probs_calib[:, 0], pos_label=0)
fpr1, tpr1, thresholds1 = roc_curve(y_true=y_calib, y_score=probs_calib[:, 1], pos_label=1)
thresh0_at_alpha = [thresholds0[np.argmin(np.abs(alpha - fpr0))] for alpha in alpha_grid]
thresh1_at_alpha = [thresholds1[np.argmin(np.abs(alpha - fpr1))] for alpha in alpha_grid]

In [10]:
probs_test = calibrated_cat_gb.predict_proba(X=X_test)

baseline_clf_outputs = []
for idx_alpha, alpha in enumerate(alpha_grid):
    # 0, 1, -1 (both), -2 (empty)
    baseline_pred_sets_outputs_alpha = []
    for i in range(probs_test.shape[0]):

        cs_at_alpha = []
        if probs_test[i, 0] < thresh0_at_alpha[idx_alpha]:
            cs_at_alpha.append(1)
        if probs_test[i, 1] < thresh1_at_alpha[idx_alpha]:
            cs_at_alpha.append(0)
        cs_at_alpha = np.array(cs_at_alpha)

        if len(cs_at_alpha) == 0:
            baseline_pred_sets_outputs_alpha.append(-2)
        elif len(cs_at_alpha) == 1:
            baseline_pred_sets_outputs_alpha.append(cs_at_alpha.item())
        elif len(cs_at_alpha) == 2:
            baseline_pred_sets_outputs_alpha.append(-1)
        else:
            print(cs_at_alpha, flush=True)
            raise ValueError("There's something wrong")
    baseline_clf_outputs.append(baseline_pred_sets_outputs_alpha)
d = {'true': y_test}
d.update({f'baselineClf_alpha{alpha_grid[j]:.3f}': o for j, o in enumerate(baseline_clf_outputs)})
results_df = pd.DataFrame(d)
results_df

true  baselineClf_alpha0.500  baselineClf_alpha0.450  \
0        0                      -2                      -2   
1        1                      -2                      -2   
2        0                       0                       0   
3        1                       1                       1   
4        0                      -2                      -2   
...    ...                     ...                     ...   
3995     0                       0                       0   
3996     0                      -2                      -2   
3997     1                       1                       1   
3998     1                      -2                      -2   
3999     1                      -2                      -2   

      baselineClf_alpha0.400  baselineClf_alpha0.350  baselineClf_alpha0.300  \
0                         -2                      -2                      -2   
1                         -2                      -2                      -2   
2                          0                       0                       0   
3                          1                       1                       1   
4                         -2                      -2                      -2   
...                      ...                     ...                     ...   
3995                       0                       0                       0   
3996                      -2                      -2                      -2   
3997                       1                       1                       1   
3998                       1                       1                       1   
3999                      -2                      -2                      -2   

      baselineClf_alpha0.250  baselineClf_alpha0.200  baselineClf_alpha0.150  \
0                         -2                      -2                      -2   
1                         -2                       1                       1   
2                          0                       0                       0   
3                          1                       1                       1   
4                          0                       0                       0   
...                      ...                     ...                     ...   
3995                       0                       0                       0   
3996                      -2                      -2                       0   
3997                       1                       1                       1   
3998                       1                       1                       1   
3999                      -2                      -2                      -2   

      baselineClf_alpha0.100  baselineClf_alpha0.050  baselineClf_alpha0.010  
0                         -2                      -2                      -1  
1                          1                       1                       1  
2                          0                       0                       0  
3                          1                       1                       1  
4                          0                       0                       0  
...                      ...                     ...                     ...  
3995                       0                       0                       0  
3996                       0                       0                       0  
3997                       1                       1                       1  
3998                       1                       1                       1  
3999                       1                       1                       1  

[4000 rows x 12 columns]

In [11]:
with open('./results/pickles/baseline_classSpecific_Clf_alpha.pkl', 'wb') as f:
    pickle.dump(results_df, f)

### General cutoff

In [17]:
probs_calib = calibrated_cat_gb.predict_proba(X=X_calib)
probs_calib_all = np.concatenate((probs_calib[y_calib == 1, 1], probs_calib[y_calib == 0, 0]))
thresh_at_alpha = np.percentile(probs_calib_all.reshape(-1, ), q=alpha_grid*100)
thresh_at_alpha

array([1.        , 1.        , 0.99656121, 0.99578849, 0.99578849,
       0.98838897, 0.98631579, 0.96573209, 0.91176471, 0.74940476,
       0.17777778])

In [18]:
probs_test = calibrated_cat_gb.predict_proba(X=X_test)

baseline_clf_outputs = []
for idx_alpha, alpha in enumerate(alpha_grid):
    # 0, 1, -1 (both), -2 (empty)
    baseline_pred_sets_outputs_alpha = []
    for i in range(probs_test.shape[0]):

        cs_at_alpha = []
        if probs_test[i, 1] >= thresh_at_alpha[idx_alpha]:
            cs_at_alpha.append(1)
        if probs_test[i, 0] >= thresh_at_alpha[idx_alpha]:
            cs_at_alpha.append(0)
        cs_at_alpha = np.array(cs_at_alpha)

        if len(cs_at_alpha) == 0:
            baseline_pred_sets_outputs_alpha.append(-2)
        elif len(cs_at_alpha) == 1:
            baseline_pred_sets_outputs_alpha.append(cs_at_alpha.item())
        elif len(cs_at_alpha) == 2:
            baseline_pred_sets_outputs_alpha.append(-1)
        else:
            print(cs_at_alpha, flush=True)
            raise ValueError("There's something wrong")
    baseline_clf_outputs.append(baseline_pred_sets_outputs_alpha)
d = {'true': y_test}
d.update({f'baselineClf_alpha{alpha_grid[j]:.3f}': o for j, o in enumerate(baseline_clf_outputs)})
results_df = pd.DataFrame(d)
results_df

true  baselineClf_alpha0.500  baselineClf_alpha0.450  \
0        0                      -2                      -2   
1        1                      -2                      -2   
2        0                       0                       0   
3        1                       1                       1   
4        0                      -2                      -2   
...    ...                     ...                     ...   
3995     0                       0                       0   
3996     0                      -2                      -2   
3997     1                       1                       1   
3998     1                      -2                      -2   
3999     1                      -2                      -2   

      baselineClf_alpha0.400  baselineClf_alpha0.350  baselineClf_alpha0.300  \
0                         -2                      -2                      -2   
1                         -2                      -2                      -2   
2                          0                       0                       0   
3                          1                       1                       1   
4                          0                       0                       0   
...                      ...                     ...                     ...   
3995                       0                       0                       0   
3996                      -2                      -2                      -2   
3997                       1                       1                       1   
3998                      -2                       1                       1   
3999                      -2                      -2                      -2   

      baselineClf_alpha0.250  baselineClf_alpha0.200  baselineClf_alpha0.150  \
0                         -2                      -2                      -2   
1                         -2                       1                       1   
2                          0                       0                       0   
3                          1                       1                       1   
4                          0                       0                       0   
...                      ...                     ...                     ...   
3995                       0                       0                       0   
3996                      -2                      -2                       0   
3997                       1                       1                       1   
3998                       1                       1                       1   
3999                      -2                      -2                      -2   

      baselineClf_alpha0.100  baselineClf_alpha0.050  baselineClf_alpha0.010  
0                         -2                      -2                      -1  
1                          1                       1                       1  
2                          0                       0                       0  
3                          1                       1                       1  
4                          0                       0                       0  
...                      ...                     ...                     ...  
3995                       0                       0                       0  
3996                       0                       0                       0  
3997                       1                       1                       1  
3998                       1                       1                       1  
3999                       1                       1                       1  

[4000 rows x 12 columns]

In [19]:
with open('./results/pickles/baseline_marginal_Clf_alpha.pkl', 'wb') as f:
    pickle.dump(results_df, f)

## Conformal (APS)

In [9]:
# Get scores
n = y_calib.shape[0]
probs_calib = cat_gb.predict_proba(X=X_calib)
cal_pi = probs_calib.argsort(axis=1)[:,::-1]
cal_srt = np.take_along_axis(probs_calib, cal_pi, axis=1).cumsum(axis=1)
cal_scores = np.take_along_axis(cal_srt, cal_pi.argsort(axis=1), axis=1)[range(n), y_calib]
# Get the score quantile
qhat = [np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, interpolation='higher') for alpha in alpha_grid]

/tmp/ipykernel_3651847/218732202.py:8: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  qhat = [np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, interpolation='higher') for alpha in alpha_grid]


In [10]:
# Deploy (output=list of length n, each element is tensor of classes)
probs_test = cat_gb.predict_proba(X=X_test)
val_pi = probs_test.argsort(1)[:,::-1]
val_srt = np.take_along_axis(probs_test, val_pi, axis=1).cumsum(axis=1)
prediction_sets = [np.take_along_axis(val_srt <= q, val_pi.argsort(axis=1), axis=1) for q in qhat]

In [24]:
conformal_outputs = []
for idx_alpha, alpha in enumerate(alpha_grid):
    # 0, 1, -1 (both), -2 (empty)
    conformal_outputs_alpha = []
    for i in range(probs_test.shape[0]):

        if prediction_sets[idx_alpha][i, :].all():
            conformal_outputs_alpha.append(-1)
        elif np.logical_not(prediction_sets[idx_alpha][i, :]).all():
            conformal_outputs_alpha.append(-2)
        elif prediction_sets[idx_alpha][i, :][0]:
            conformal_outputs_alpha.append(0)
        elif prediction_sets[idx_alpha][i, :][1]:
            conformal_outputs_alpha.append(1)
        else:
            print(prediction_sets[idx_alpha][i, :], flush=True)
            raise ValueError("There's something wrong")
    conformal_outputs.append(conformal_outputs_alpha)
d = {'true': y_test}
d.update({f'conformal_aps_alpha{alpha_grid[j]:.3f}': o for j, o in enumerate(conformal_outputs)})
results_df = pd.DataFrame(d)
results_df

true  conformal_aps_alpha0.500  conformal_aps_alpha0.450  \
0        0                         1                         1   
1        1                         1                         1   
2        0                        -2                        -2   
3        1                        -2                        -2   
4        0                         0                         0   
...    ...                       ...                       ...   
3995     0                        -2                         0   
3996     0                         0                         0   
3997     1                         1                         1   
3998     1                         1                         1   
3999     1                         1                         1   

      conformal_aps_alpha0.400  conformal_aps_alpha0.350  \
0                            1                         1   
1                            1                         1   
2                           -2                         0   
3                           -2                        -2   
4                            0                         0   
...                        ...                       ...   
3995                         0                         0   
3996                         0                         0   
3997                         1                         1   
3998                         1                         1   
3999                         1                         1   

      conformal_aps_alpha0.300  conformal_aps_alpha0.250  \
0                            1                         1   
1                            1                         1   
2                            0                         0   
3                           -2                        -2   
4                            0                         0   
...                        ...                       ...   
3995                         0                         0   
3996                         0                         0   
3997                         1                         1   
3998                         1                         1   
3999                         1                         1   

      conformal_aps_alpha0.200  conformal_aps_alpha0.150  \
0                            1                         1   
1                            1                         1   
2                            0                         0   
3                           -2                         1   
4                            0                         0   
...                        ...                       ...   
3995                         0                         0   
3996                         0                         0   
3997                         1                         1   
3998                         1                         1   
3999                         1                         1   

      conformal_aps_alpha0.100  conformal_aps_alpha0.050  \
0                            1                         1   
1                            1                         1   
2                            0                         0   
3                            1                         1   
4                            0                         0   
...                        ...                       ...   
3995                         0                         0   
3996                         0                         0   
3997                         1                         1   
3998                         1                         1   
3999                         1                         1   

      conformal_aps_alpha0.010  
0                           -1  
1                           -1  
2                           -1  
3                           -1  
4                           -1  
...                        ...  
3995                        -1  
3996                        -1  
3997                        -1  

In [25]:
with open('./results/pickles/conformal_aps_Clf_alpha.pkl', 'wb') as f:
    pickle.dump(results_df, f)